将变量取值和计算图结构分成不同的文件存储有时候也不方便，于是Tensorflow提供convert_variables_to_constants函数，通过这个函数可以将计算图中的变量机器取值通过常量的方式保存，这个整个tensorflow计算图可以统一保存在一个文件中 

In [2]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [3]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2


In [4]:
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    #导出当前计算图的GraphDef部分，只需要这一部分就可以完成从输入层到输出层的计算过程
    graph_def = tf.get_default_graph().as_graph_def()

    #如果只关心程序中定义的某些计算时，和这些计算无关的节点就没有必要导出并保存了，在下面一行代码中，最后一个参数【‘add’】给出了需要保存的节点名称。add节点时上面定义的两个变量相加的操作
    #注意这里给出的时计算节点的名称，所以没有给出后面的：0
    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph_def, ['add']
    )
    #将导出的模型存入文件
    with tf.gfile.GFile("./path/to/model/combined_model.pb", "wb") as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


下面的程序可以直接计算定义的加法运算的结果。只需要得到计算图中某个节点的取值时，这提供了一个更加方法的方法

In [5]:
import tensorflow as tf
from tensorflow.python.platform import gfile

In [6]:
with tf.Session() as sess:
    model_filename = "./path/to/model/combined_model.pb"
    #读取保存的模型文件，并将文件解析成对应的GraphDef Protocol Buffer
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    #将graph_def 中保存的图加载到当前的图中。return_elements=["add:0"]给出了返回张量的名称，在保存的时候给出的时计算节点的名称，所以为“add”。在加载的时候给出的时张量的名称，所以时add:0
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([ 3.], dtype=float32)]


In [2]:
v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)

#通过使用variable_to_restore函数可以直接生成上面代码中提供的字典
print(ema.variables_to_restore())

{'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>}


In [5]:
saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, "./path/to/model/model1.ckpt")
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from ./path/to/model/model1.ckpt
0.0999999
